In [16]:
# -*- coding: utf-8 -*-
"""blip2-opt-2.7b.ipynb"""

!pip install -q transformers torch accelerate pillow openpyxl tqdm

import os
import re
import json
import random
import time
from datetime import datetime
import pandas as pd
import torch
from tqdm import tqdm
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from huggingface_hub import notebook_login
from google.colab import drive

In [2]:
# ---------------- Mount Drive ----------------
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# ---------------- Configuration ----------------
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)

HF_TOKEN = ""   # your token
MODEL_ID = "Salesforce/blip2-opt-2.7b"
DTYPE = torch.float16

os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
notebook_login()

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Running on {device.upper()}")

✅ Running on CUDA


In [5]:
# ---------------- Paths ----------------
DATA_ROOT = "/content/drive/MyDrive/Project/test_folder"
TF_JSON = os.path.join(DATA_ROOT, "TF_data.json")
MC_JSON = os.path.join(DATA_ROOT, "MC_data.json")

DETAILED_RESULTS_XLSX = os.path.join(DATA_ROOT, "blip2_detailed_results.xlsx")
SUMMARY_RESULTS_XLSX  = os.path.join(DATA_ROOT, "blip2_summary_results.xlsx")

assert os.path.exists(TF_JSON), f"TF JSON not found: {TF_JSON}"
assert os.path.exists(MC_JSON), f"MC JSON not found: {MC_JSON}"


In [17]:
# ---------------- Normalization Helpers ----------------
def normalize_tf(pred):
    p = str(pred).strip().lower()
    if re.fullmatch(r"(true|t)", p): return "True"
    if re.fullmatch(r"(false|f)", p): return "False"
    if "true" in p and "false" not in p: return "True"
    if "false" in p and "true" not in p: return "False"
    return "True" if "yes" in p else ("False" if "no" in p else "False")

def normalize_mc(pred, choices=None):
    if not pred: return ""
    p = str(pred).strip().upper()
    m = re.search(r"\b([A-F])\b", p)
    if m: return m.group(1)
    if choices:
        for idx, c in enumerate(choices):
            if c and c.strip().lower() in p.lower():
                return chr(ord("A") + idx)
    return p[:1] if p else ""

def build_prompt(qtype, question, choices=None):
    if qtype == "TF":
        return (
            "Example 1: Net profit rose in 2020 vs 2019? <answer>False</answer>\n"
            "Example 2: Expenses decreased in 2022 vs 2021? <answer>True</answer>\n"
            "<start_of_image>\n"
            "You are given a chart image and a True/False question.\n"
            "Respond ONLY with 'True' or 'False' inside <answer> tags.\n\n"
            f"Question: {question}\n<answer>"
        )
    elif qtype == "MC":
        example = (
            "Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</answer>\n"
            "Example 2: Which product had lowest cost? Choices: A.P1 B.P2 C.P3 D.P4 <answer>D</answer>\n"
            "Example 3: Which region grew fastest? Choices: A.Asia B.Europe C.US D.Africa <answer>B</answer>\n"
        )
        choices_str = "\n".join(f"{chr(ord('A')+i)}. {c}" for i, c in enumerate(choices or []))
        return (
            example
            + "<start_of_image>\n"
            "You are given a chart image and a multiple‐choice question.\n"
            "Respond ONLY with the letter inside <answer> tags.\n\n"
            f"Question: {question}\nChoices:\n{choices_str}\n<answer>"
        )
    else:
        raise ValueError(f"Unsupported question type: {qtype}")

def extract_final_answer(raw_output: str, choices=None):
    if not isinstance(raw_output, str) or not raw_output.strip():
        return ""
    text = raw_output.strip()
    m_tag = re.findall(r"<\s*answer\s*>\s*([A-Z0-9TrueFalse]+?)\s*<\s*/\s*answer\s*>", text, flags=re.IGNORECASE)
    if m_tag:
        ans = m_tag[-1].strip().capitalize()
        return ans
    if choices:
        for idx, c in enumerate(choices):
            if c and c.strip().lower() in text.lower():
                return chr(ord('A') + idx)
    m_fallback = re.findall(r"\b(True|False|[A-F])\b", text, flags=re.IGNORECASE)
    if m_fallback:
        return m_fallback[-1].strip().capitalize()
    return ""


In [25]:
# ---------------- Dataset Loader ----------------
def load_items(path, qtype, subfolder):
    with open(path, "r", encoding="utf-8") as f:
        items = json.load(f)
    return [
        {
            "type":      qtype,
            "question":  it["question"],
            "answer":    it["answer"],
            "choices":   (it.get("choices") if qtype=="MC" else None),
            "image_path": os.path.join(DATA_ROOT, subfolder, it["image"])
        }
        for it in items
    ]

def load_local_dataset(num_mc=40, num_tf=40):
    mc = load_items(MC_JSON, "MC", "MC_images")
    tf = load_items(TF_JSON, "TF", "TF_images")
    data = mc[:num_mc] + tf[:num_tf]
    random.shuffle(data)
    return data

In [19]:
# ---------------- Model Loading ----------------
print("🔄 Loading BLIP-2 OPT-2.7B …")
processor = Blip2Processor.from_pretrained(MODEL_ID)
model = Blip2ForConditionalGeneration.from_pretrained(
    MODEL_ID,
    torch_dtype=DTYPE,
    device_map="auto"
).eval()
print("✅ Model loaded on:", device)


🔄 Loading BLIP-2 OPT-2.7B …


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded on: cuda


In [26]:
# ---------------- Inference ----------------
@torch.no_grad()
def infer_blip2(image_path, question, qtype, choices=None):
    prompt = build_prompt(qtype, question, choices)
    image  = Image.open(image_path).convert("RGB")
    # Print original image size
    orig_w, orig_h = image.size
    print(f"🖼️ Original image size: {orig_w}×{orig_h}")
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device, DTYPE)
    out_ids = model.generate(**inputs, max_new_tokens=128)
    answer  = processor.decode(out_ids[0], skip_special_tokens=True)
    return answer.strip()

def accuracy(preds, gts):
    return sum(str(p).strip() == str(gt).strip() for p, gt in zip(preds, gts)) / len(preds) if preds else 0


In [27]:
# ---------------- Evaluation ----------------
def evaluate_and_summarize(examples):
    rows, preds, gts, types = [], [], [], []
    print("🚀 Running inference …")
    start_time = time.time()

    for idx, ex in enumerate(tqdm(examples, desc="Evaluating", unit="example")):
        raw = infer_blip2(ex["image_path"], ex["question"], ex["type"], ex.get("choices"))
        if ex["type"] == "MC":
            extracted = extract_final_answer(raw, ex.get("choices"))
        else:
            extracted = extract_final_answer(raw, None)

        print(f"\n📄 Example {idx+1}/{len(examples)}")
        print(f"🖼️ Image: {os.path.basename(ex['image_path'])}")
        print(f"❓ Question: {ex['question']}")
        print(f"RAW OUTPUT: {raw}")
        print(f"EXTRACTED ANSWER: {extracted}\n")

        if ex["type"] == "MC":
            norm = normalize_mc(extracted, ex.get("choices"))
        else:
            norm = normalize_tf(extracted)

        rows.append({
            "image_path": ex["image_path"],
            "type":       ex["type"],
            "question":   ex["question"],
            "ground_truth": ex["answer"],
            "prediction_by_model": norm,
            "raw_output": raw,
            "raw_output_2": (raw[:200] + "..." if len(raw)>200 else raw),
            "raw_output_length": len(raw)
        })
        preds.append(norm)
        gts.append(ex["answer"])
        types.append(ex["type"])

    total_time_sec = time.time() - start_time
    print(f"⌛ Total time taken: {total_time_sec:.2f} seconds")

    summary = {
        "model_name":      MODEL_ID,
        "MC_Score":        round(accuracy([p for p,t in zip(preds,types) if t=="MC"],
                                         [g for g,t in zip(gts,types) if t=="MC"]), 4),
        "TF_Score":        round(accuracy([p for p,t in zip(preds,types) if t=="TF"],
                                         [g for g,t in zip(gts,types) if t=="TF"]), 4),
        "Weighted_Avg":    round(accuracy(preds, gts), 4),
        "Total_Questions": len(preds),
        "Total_Time_Sec":  total_time_sec,
        "Timestamp":        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }

    print("✅ Inference completed!")
    print(f"📊 MC Accuracy: {summary['MC_Score']}")
    print(f"📊 TF Accuracy: {summary['TF_Score']}")
    print(f"📊 Weighted Avg: {summary['Weighted_Avg']}")

    return pd.DataFrame(rows), summary


In [28]:
# ---------------- Main Execution ----------------
if __name__ == "__main__":
    examples = load_local_dataset(num_mc=40, num_tf=40)
    print(f"Total examples: {len(examples)}")
    df_details, summary = evaluate_and_summarize(examples)

    timestamp     = datetime.now().strftime("%Y%m%d_%H%M%S")
    detailed_path = DETAILED_RESULTS_XLSX.replace(".xlsx", f"_{timestamp}.xlsx")
    df_details.to_excel(detailed_path, index=False)
    print("✅ Saved detailed results:", detailed_path)

    df_sum = pd.DataFrame([summary])
    if os.path.exists(SUMMARY_RESULTS_XLSX):
        old   = pd.read_excel(SUMMARY_RESULTS_XLSX)
        df_sum = pd.concat([old, df_sum], ignore_index=True)
    df_sum.to_excel(SUMMARY_RESULTS_XLSX, index=False)
    print("✅ Saved summary results:", SUMMARY_RESULTS_XLSX)


Total examples: 80
🚀 Running inference …


Evaluating:   1%|▏         | 1/80 [00:00<00:14,  5.36example/s]

🖼️ Original image size: 785×826

📄 Example 1/80
🖼️ Image: 1246069785_3_crop_0.jpg
❓ Question: The Global Oncotype DX Product Revenue increased every year from 2008 to 2013.
RAW OUTPUT: Example 1: Net profit rose in 2020 vs 2019? <answer>False</answer>
Example 2: Expenses decreased in 2022 vs 2021? <answer>True</answer>
<start_of_image>
You are given a chart image and a True/False question.
Respond ONLY with 'True' or 'False' inside <answer> tags.

Question: The Global Oncotype DX Product Revenue increased every year from 2008 to 2013.
<answer>False</answer>
EXTRACTED ANSWER: False

🖼️ Original image size: 1845×862


Evaluating:   4%|▍         | 3/80 [00:00<00:14,  5.25example/s]


📄 Example 2/80
🖼️ Image: 1246868763_15_crop_0.jpg
❓ Question: Which brand experienced the largest increase in units from 2009 to Q3 2014 TTM?
RAW OUTPUT: Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</answer>
Example 2: Which product had lowest cost? Choices: A.P1 B.P2 C.P3 D.P4 <answer>D</answer>
Example 3: Which region grew fastest? Choices: A.Asia B.Europe C.US D.Africa <answer>B</answer>
<start_of_image>
You are given a chart image and a multiple‐choice question.
Respond ONLY with the letter inside <answer> tags.

Question: Which brand experienced the largest increase in units from 2009 to Q3 2014 TTM?
Choices:
A. A
B. B
C. C
D. D
<answer>C</answer>
EXTRACTED ANSWER: C

🖼️ Original image size: 775×864

📄 Example 3/80
🖼️ Image: 1245845967_7_crop_0.jpg
❓ Question: Based on the chart, what is the most common outcome among the subjects?
RAW OUTPUT: Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</ans

...

Evaluating:  99%|█████████▉| 79/80 [00:15<00:00,  5.25example/s]


📄 Example 78/80
🖼️ Image: 1245561843_11_crop_0.jpg
❓ Question: Which category experienced the largest decrease in units under construction from 1Q12 to 3Q14?
RAW OUTPUT: Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</answer>
Example 2: Which product had lowest cost? Choices: A.P1 B.P2 C.P3 D.P4 <answer>D</answer>
Example 3: Which region grew fastest? Choices: A.Asia B.Europe C.US D.Africa <answer>B</answer>
<start_of_image>
You are given a chart image and a multiple‐choice question.
Respond ONLY with the letter inside <answer> tags.

Question: Which category experienced the largest decrease in units under construction from 1Q12 to 3Q14?
Choices:
A. A
B. B
C. C
D. D
<answer>C</answer>
EXTRACTED ANSWER: C

🖼️ Original image size: 627×758

📄 Example 79/80
🖼️ Image: 1245806271_26_crop_0.jpg
❓ Question: The OEM 2014 Light Truck Fleet Performance for manufacturer D exceeds the 2025 Light Truck Target of 43.8 MPG.
RAW OUTPUT: Example 1: Net profit 

Evaluating: 100%|██████████| 80/80 [00:15<00:00,  5.18example/s]


📄 Example 80/80
🖼️ Image: 1246170879_25_crop_0.jpg
❓ Question: What was the trend in revenue from 2009 to 2013 as shown in the chart?
RAW OUTPUT: Example 1: Which year had highest revenue? Choices: A.2018 B.2019 C.2020 D.2021 <answer>A</answer>
Example 2: Which product had lowest cost? Choices: A.P1 B.P2 C.P3 D.P4 <answer>D</answer>
Example 3: Which region grew fastest? Choices: A.Asia B.Europe C.US D.Africa <answer>B</answer>
<start_of_image>
You are given a chart image and a multiple‐choice question.
Respond ONLY with the letter inside <answer> tags.

Question: What was the trend in revenue from 2009 to 2013 as shown in the chart?
Choices:
A. A
B. B
C. C
D. D
<answer>A</answer>
EXTRACTED ANSWER: A

⌛ Total time taken: 15.44 seconds
✅ Inference completed!
📊 MC Accuracy: 0.25
📊 TF Accuracy: 0.55
📊 Weighted Avg: 0.4
✅ Saved detailed results: /content/drive/MyDrive/Project/test_folder/blip2_detailed_results_20251110_171532.xlsx


✅ Saved summary results: /content/drive/MyDrive/Project/test_folder/blip2_summary_results.xlsx
